Importacion de librarias

In [1]:
import numpy as np
import pandas as pd
from pandas import *
import re
import math

Importando archivos de excel

In [2]:
file = ('segundoejercicio.xlsx')
file2 = ('butacas.xlsx')
repository_data = pd.read_excel(file, skiprows= 6, usecols=[0,1,2,3,4])
repository_butacas = pd.read_excel(file2)

Eliminando NAN (empty colums), eliminando las files que contienen totales

In [3]:
#remove NAN (empty rows)
filter_valid_rows = repository_data.dropna(how='all')
filter_butacas = repository_butacas.dropna(how='all')


#remove totals
filter_valid_rows = filter_valid_rows.drop(filter_valid_rows[filter_valid_rows['Film'] == 'Total'].index)
filter_valid_rows = filter_valid_rows.drop(filter_valid_rows[filter_valid_rows['Screen'] == 'Total'].index)
filter_valid_rows = filter_valid_rows.drop(filter_valid_rows[filter_valid_rows['Cinema'] == 'Grand Total'].index)

LLenar los campos NAN de Screen y Cinema.

In [4]:
last_screen = 0
last_cinema = 0
remove_index_list = []
for index, row in filter_valid_rows.iterrows():
    #change NAN value
    try:
        screen = float( row['Screen'] )
        if( math.isnan( screen ) ):
            filter_valid_rows.at[index,'Screen'] = last_screen
        else:
            last_screen = screen

    except:
        remove_index_list = remove_index_list + [index]
   
    # remove cinema name. 
    #change NAN
    try:
        #remove text
        cinema = re.findall('[0-9]+', row['Cinema'])
        last_cinema = (cinema[0])
        filter_valid_rows.at[index,'Cinema'] = last_cinema
        
    except:
        filter_valid_rows.at[index,'Cinema'] = last_cinema
  

Eliminacion Subtitulos y calida de la pelicula. ( borrado de parentesis )

In [5]:
filter_valid_rows["Film"] = filter_valid_rows["Film"].str.split("\(").str.get(0)

Se agregan las butacas

In [6]:
#set by defaullt
filter_valid_rows = filter_valid_rows.assign(butacas = 252 ) 
filter_valid_rows = filter_valid_rows.astype({'Cinema': float, 'Screen': float})


In [7]:
#comparar el cinema y las salas para ver con cuantos asientos cuenta
for index, row in filter_valid_rows.iterrows():
    for index2, row2 in filter_butacas.iterrows():
        try:
            if(row['Cinema'] == row2['Theatre'] and row['Screen'] == row2['Screen'] ):
                filter_valid_rows.at[index,'butacas'] = row2['Seats']
        except:
            print('ouch')


In [8]:
filter_valid_rows

,Cinema,Screen,Film,Number of Sessions,Admits,butacas
0,2200.0,1.0,Batman,48.0,1830.0,252
1,2200.0,1.0,Batman,18.0,419.0,252
2,2200.0,1.0,Muestrame al Padre,1.0,84.0,252
3,2200.0,1.0,Los tipos Malos,5.0,51.0,252
4,2200.0,1.0,Morbius,2.0,37.0,252
...,...,...,...,...,...,...
1388,795.0,10.0,Bachchhan Paandey,3.0,164.0,124
1389,795.0,10.0,El Exorcismo,5.0,46.0,124
1390,795.0,10.0,Casate Conmigo,4.0,37.0,124
1391,795.0,10.0,Morbius,3.0,16.0,124


Se calcula la columna maxbutacas

In [9]:
#calculate maxButacas
filter_valid_rows['maxbutacas'] = filter_valid_rows['Number of Sessions'] * filter_valid_rows['butacas']

In [10]:
filter_valid_rows
filter_valid_rows.to_excel('resultado_final.xlsx',sheet_name='parcial')

Calculo de columna Occ 

In [11]:
movies = filter_valid_rows.groupby("Film").agg(sum)
movies.reset_index(inplace=True)
movies['Occ'] =  round( (movies['Admits'] / movies['maxbutacas'] ) *100 , 0).astype(int)

Eliminando Columnas en desuso

In [12]:
#remove colums
movies.drop(['Number of Sessions','butacas', 'Cinema','Screen'], axis = 1, inplace = True) 

Agregando formato de salida

In [13]:
movies = movies.astype({'Film': str, 'Admits': int, 'maxbutacas': int, 'Occ': str})

In [14]:
movies['Occ'] = movies['Occ']+'%'
movies

,Film,Admits,maxbutacas,Occ
0,Ambulance,852,6880,12%
1,Ambulancia Plan de Huida,5300,84196,6%
2,BTS Permission to Dance,6023,8462,71%
3,Bachchhan Paandey,518,1278,41%
4,Batman,208188,1188080,18%
5,Belfast,1074,15870,7%
6,Blacklight,513,4496,11%
7,Blue Bayou,5,262,2%
8,Caceria de Brujas,323,3881,8%
9,Callejon de las almas perdidas,106,3127,3%


In [15]:
movies.to_excel('resultado_final.xlsx',sheet_name='parcial')